In [11]:
import os
import requests
from tools import quadratic_calculator, fib_list, convert_currency, query_library, search_web
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate


load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [12]:
#initialize LLM 
llm = ChatOpenAI(
    model="gpt-5-nano"
)

In [13]:
#bind llm to tools
llm_with_tools = llm.bind_tools([quadratic_calculator, fib_list, convert_currency, query_library, search_web])

FUNCTION_NAMES = {
    "quadratic_calculator": quadratic_calculator,
    "fib_list": fib_list,
    "convert_currency": convert_currency,
    "query_library": query_library,
    "search_web": search_web
}


In [14]:
#generate the prompt 
prompt = ChatPromptTemplate([
    ("system", "You are a helpful agent that will be given a set of tools. Your only job is to execute the right one"),
    ("user", "{query}")
])

In [ ]:
query = "Search my library and summarize the plot of 1984. What happens at the end?"
#query = "Search the web and recommend me the best skin care product for men"
#query = "can you help me find the solution for x**2 + 8x + 16?"
#query = "what are the first 20 numbers in the fibonacci sequence?"
#query = "how much is 10000000000 us dollars in us dollars?"

res = llm_with_tools.invoke(prompt.format_messages(query=query)).tool_calls

In [ ]:
print(res)

In [17]:
#make sure tool was called
if not res:
    raise ValueError("No tool called")


#figure out which tool was called
fn = FUNCTION_NAMES[res[0]["name"]]

#call the tool 
ans = fn.invoke(res[0]["args"])

In [18]:
print(ans)

There isn’t a universal “best”—it depends on your skin type and goals. Here are solid, well-regarded options from the results:

- Kiehl’s Men’s Best Sellers: a versatile starter range (cleansers, shaving gel, moisturizers) with essential oils. Great if you want a straightforward, proven lineup.
- Brickell Men’s Products: natural, anti-aging creams (e.g., DMAE-based) that suit many skin types and emphasize clean ingredients.
- Lab Series Daily Rescue: energizing liquid moisturizer that hydrates deeply and supports the skin barrier—good for daily use and protection.

Quick routine (for most men):
- Cleanser (AM/PM) tailored to your skin type.
- Moisturizer with SPF in the morning; consider a hydrating option at night.
- Optional: vitamin C serum in the morning and/or eye cream when needed.

If you share your skin type (oily, dry, sensitive, mature) and budget, I’ll tailor a specific product pick and routine.
